## Plot three different spectra from the BOSZ stellar library: roughly equivalent to an O, G, and M dwarf.

First we read in the data files, we need to take them directly from a URL because they are not in Portal.

In [ ]:
from astronify.series import SoniSeries
from astropy.io import fits
from astropy.table import QTable
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import time

In [ ]:
url_root = "http://archive.stsci.edu/missions/hlsp/bosz/fits/insbroad_000200/"
files_to_read = [
    url_root + "metal_+0.00/carbon_+0.00/alpha_+0.00/amp00cp00op00t3500g45v20modrt0b200rs.fits",
    url_root + "metal_+0.00/carbon_+0.00/alpha_+0.00/amp00cp00op00t5750g45v20modrt0b200rs.fits",
    url_root + "metal_+0.00/carbon_+0.00/alpha_+0.00/amp00cp00op00t25000g45v20modrt0b200rs.fits"]
stypes = ["M star", "G star", "O star"]

In [ ]:
# Read in each FITS file and make a static, visual plot.  Play the sonification.
for file, stype in zip(files_to_read, stypes):
    with fits.open(file, mode="readonly") as hdulist:
        data = hdulist[1].data
        
    wls = data['Wavelength']
    surface_brightnesses = data['SpecificIntensity']
    fluxes = math.pi * surface_brightnesses
    
    # Extract the temperature value from the name of the model file.
    teff_val = file.split('/')[-1][14:18]
    # BOSZ stores high temperature as 4 digits even though they are actually five digits...
    if teff_val == '2500':
        teff_val = '25000'
    
    # Keep only the data from 0.8 to 1.2 microns (8000 to 12000 Angstroms)
    # Wavelengths in the BOSZ files are in Angstroms.
    where_keep = np.where((wls >= 8000) & (wls <= 12000))[0]
    wls = wls[where_keep]
    fluxes = fluxes[where_keep]
    
    plt.plot(wls, fluxes, '-k')
    plt.title("T_eff = " + teff_val)
    plt.show()
    
    # First we make an Astropy table of wavelength and flux to send to SoniSeries.
    os.system("say " + "Sonification for type " + stype)
    data_table = QTable([wls, fluxes], names=('Wavelength', 'Flux'))
    soni_obj = SoniSeries(data_table, time_col="Wavelength", val_col="Flux")
    soni_obj.sonify()
    soni_obj.play()
    time.sleep(2)